In [1]:
!pip install -q presidio-analyzer presidio-anonymizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.7 MB/s eta 0:00:00


In [2]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

# Create analyzer and anonymizer
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# Romanian text
text = """
Numele meu este Andrei Popescu și locuiesc pe Strada Libertății 45, București.
Adresa mea de email este andrei.popescu@example.ro și numărul de telefon este +40 722 123 456.
M-am născut pe 14.03.1990, iar CNP-ul meu este 1960314330018.
Cardul meu bancar are numărul 4532 1234 5678 9012.
"""

# Detect multiple entity types
results = analyzer.analyze(
    text=text,
    entities=[
        "PERSON", "LOCATION", "EMAIL_ADDRESS", "PHONE_NUMBER",
        "DATE_TIME", "CREDIT_CARD"  # CNP is not built-in, will need custom recognizer
    ],
    language="en"  # Presidio mostly works with English, but regex entities (phone, credit card, etc.) still work
)

# Anonymize text
anonymized_text = anonymizer.anonymize(
    text=text,
    analyzer_results=results
)

print("Original text:\n", text)
print("\nDetected entities:")
for r in results:
    print(r, text[r.start:r.end])

print("\nAnonymized text:\n", anonymized_text.text)

Original text:
 
Numele meu este Andrei Popescu și locuiesc pe Strada Libertății 45, București.
Adresa mea de email este andrei.popescu@example.ro și numărul de telefon este +40 722 123 456.
M-am născut pe 14.03.1990, iar CNP-ul meu este 1960314330018.
Cardul meu bancar are numărul 4532 1234 5678 9012.


Detected entities:
type: EMAIL_ADDRESS, start: 105, end: 130, score: 1.0 andrei.popescu@example.ro
type: PERSON, start: 1, end: 34, score: 0.85 Numele meu este Andrei Popescu și
type: DATE_TIME, start: 190, end: 200, score: 0.6 14.03.1990
type: PHONE_NUMBER, start: 158, end: 173, score: 0.4 +40 722 123 456

Anonymized text:
 
<PERSON> locuiesc pe Strada Libertății 45, București.
Adresa mea de email este <EMAIL_ADDRESS> și numărul de telefon este <PHONE_NUMBER>.
M-am născut pe <DATE_TIME>, iar CNP-ul meu este 1960314330018.
Cardul meu bancar are numărul 4532 1234 5678 9012.

